<a href="https://colab.research.google.com/github/Apple03244/Colaboratory/blob/main/Funtional_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import google.colab as colab
fm=colab.files
files_=fm.upload()

Saving train.csv to train (1).csv


In [45]:
import pandas as pd
data=pd.read_csv('/content/train.csv')

In [46]:
import sklearn.model_selection as skmod
import sklearn.compose as skcom
import sklearn.impute as skimp
import sklearn.preprocessing as skpre
import sklearn.pipeline as skpip
import numpy as np

## Funtional method
- 이전 방법들은 "정적인" 방법으로 모델의 구조를 이해하고 copy하기 쉽다는 장점이 있다.
- 하지만 동적인 method 와 달리 모델을 변경하기 어려우므로 아해의 방법을 이해해두자

In [47]:
data["일시"]=data["일시"].apply(lambda x:pd.to_datetime(x))

In [48]:
imp=skimp.SimpleImputer(strategy="constant",fill_value=0)
data[data.columns.difference(["일시","평균기온"])]=imp.fit_transform(data[data.columns.difference(["일시","평균기온"])])

### 모델 만들기

In [49]:
import tensorflow as ts
import tensorflow.keras as tk

In [62]:
class WideAndDeepModel(tk.Model):
  def __init__(self,units=30,activation='relu',**kwargs):
    super().__init__(**kwargs) # 모델에 이름을 부여하기 위해 필요
    self.norm_layer_wide=tk.layers.Normalization()
    self.norm_layer_deep=tk.layers.Normalization()
    self.deep_1_layer=tk.layers.Dense(units,activation=activation)
    self.deep_2_layer=tk.layers.Dense(units,activation=activation)
    self.output_layer=tk.layers.Dense(1)
    self.sub_output_layer=tk.layers.Dense(1)

  def call(self,inputs):
    wide_input,deep_input=inputs
    norm_wide=self.norm_layer_wide(wide_input)
    norm_deep=self.norm_layer_deep(deep_input)
    deep_1=self.deep_1_layer(norm_deep)
    deep_2=self.deep_2_layer(deep_1)
    concate=tk.layers.concatenate([norm_wide,deep_2])
    output=self.output_layer(concate)
    sub_output=self.sub_output_layer(concate)
    return (output,sub_output)

In [63]:
MyModel=WideAndDeepModel(30,activation="relu",name="MyFirstModel")

### 모델 학습하기

In [64]:
input_wide=data[["최고기온","최저기온"]]
input_deep=data[data.columns.difference(["최고기온","최저기온","평균기온","일시"])]
inputs=(input_wide,input_deep)

In [65]:
MyModel.norm_layer_wide.adapt(input_wide)
MyModel.norm_layer_deep.adapt(input_deep)

In [66]:
# compile
MyModel.compile(optimizer=tk.optimizers.Adam(learning_rate=0.01),
                loss=tk.losses.mae,
                metrics=["RootMeanSquaredError"])

In [71]:
test_output=MyModel.call(inputs=(input_wide,input_deep))